In [7]:
import copy
import time
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Subset, DataLoader
from models_utils.Datasets import *
from models_utils.GLOBALS import *
import torch.nn as nn
import torch.nn.functional as F

In [8]:
# get train data
train_data = pd.read_csv('train.csv', index_col=0)
train_data['activity'] = train_data['activity'].map(activity_id_mapping)
data_type_1 = train_data[train_data['sensor'] == 'vicon'].reset_index()
data_type_2 = train_data[train_data['sensor'] == 'smartwatch'].reset_index()

In [9]:
# CNN / LSTM feature extractor
class MultivariateCNNLSTM(nn.Module):
    def __init__(self, num_channels, input_length, num_classes=18, lstm_hidden_size=256, lstm_layers=1):
        super(MultivariateCNNLSTM, self).__init__()
        self.num_channels = num_channels
        self.input_length = input_length
        self.num_classes = num_classes
        self.lstm_hidden_size = lstm_hidden_size

        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        output_size_after_conv2 = input_length // 4
        self.lstm_input_size = 128

        # LSTM layer
        self.lstm = nn.LSTM(input_size=self.lstm_input_size, hidden_size=lstm_hidden_size, num_layers=lstm_layers,
                            batch_first=True)

        # output size from LSTM to first fc layer
        self.fc1_size = lstm_hidden_size * output_size_after_conv2
        self.fc1 = nn.Linear(self.fc1_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # reshape x  for LSTM
        x = x.permute(0, 2, 1)

        x, (hn, cn) = self.lstm(x)

        # flatten the output for fc layer
        x = x.contiguous().view(-1, self.fc1_size)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [25]:
# train first model
target_size_type1 = 3000
whole_dataset_type1 = TrainDataframeWithLabels(data_type_1, '1', target_size_type1)
model_CNN_LSTM_type1 = MultivariateCNNLSTM(3, target_size_type1, 18).to(device)

labels = data_type_1['activity'].to_list()
train_idx, val_idx = train_test_split(
    range(len(whole_dataset_type1)),
    test_size=0.2, 
    stratify=labels,
)

# create stratified datasets
train_dataset = Subset(whole_dataset_type1, train_idx)
val_dataset = Subset(whole_dataset_type1, val_idx)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_CNN_LSTM_type1.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

best_val_loss = float('inf') 
best_model_state = copy.deepcopy(model_CNN_LSTM_type1.state_dict())  

num_epochs = 30
for epoch in range(num_epochs):
    epoch_start_time = time.time()

    total_train_loss = 0
    train_correct = 0
    train_total = 0

    total_val_loss = 0
    val_correct = 0
    val_total = 0

    # training 
    model_CNN_LSTM_type1.train() 
    for batch_idx, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        normalized_inputs = (inputs - min_values_type1) / (max_values_type1 - min_values_type1 + 1e-6)
        normalized_inputs = normalized_inputs.transpose(1, 2)
        optimizer.zero_grad()
        outputs = model_CNN_LSTM_type1(normalized_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        total_train_loss += loss.item()

    # validation Phase
    model_CNN_LSTM_type1.eval()  
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            normalized_inputs = (inputs - min_values_type1) / (max_values_type1 - min_values_type1 + 1e-6)
            normalized_inputs = normalized_inputs.transpose(1, 2)
            outputs = model_CNN_LSTM_type1(normalized_inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            total_val_loss += loss.item()

    epoch_end_time = time.time() 
    epoch_duration = epoch_end_time - epoch_start_time 

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_train_loss / len(train_dataloader)
    val_accuracy = 100 * val_correct / val_total
    avg_val_loss = total_val_loss / len(val_dataloader)
    scheduler.step(avg_val_loss)

    # check if this is the best model based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model_CNN_LSTM_type1.state_dict())
        # save the best model to disk
        torch.save(best_model_state, 'Type1CNNLSTMModel.pth')
        print(f'Saving Best Model with loss: {avg_val_loss}')

    print("--------------")
    print(f'Epoch [{epoch + 1}/{num_epochs}], Epoch Duration: {epoch_duration:.2f} seconds')
    print(
        f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(
        f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

model_CNN_LSTM_type1 = MultivariateCNNLSTM(3, target_size_type1, 18).to(device)
model_CNN_LSTM_type1.load_state_dict(torch.load('Type1CNNLSTMModel.pth'))

Saving Best Model with loss: 1.9149110588160427
--------------
Epoch [1/30], Epoch Duration: 31.54 seconds
Epoch [1/30], Training Loss: 2.2161, Training Accuracy: 16.98%
Epoch [1/30], Validation Loss: 1.9149, Validation Accuracy: 19.55%
Saving Best Model with loss: 1.709758387370543
--------------
Epoch [2/30], Epoch Duration: 32.59 seconds
Epoch [2/30], Training Loss: 1.8019, Training Accuracy: 24.83%
Epoch [2/30], Validation Loss: 1.7098, Validation Accuracy: 26.48%
Saving Best Model with loss: 1.6206989532167262
--------------
Epoch [3/30], Epoch Duration: 32.39 seconds
Epoch [3/30], Training Loss: 1.6337, Training Accuracy: 32.18%
Epoch [3/30], Validation Loss: 1.6207, Validation Accuracy: 33.49%
--------------
Epoch [4/30], Epoch Duration: 31.88 seconds
Epoch [4/30], Training Loss: 1.5354, Training Accuracy: 37.52%
Epoch [4/30], Validation Loss: 1.6332, Validation Accuracy: 32.99%
Saving Best Model with loss: 1.4482392858375202
--------------
Epoch [5/30], Epoch Duration: 30.79 se

FileNotFoundError: [Errno 2] No such file or directory: 'Type1CNNLSTMModel.pth'

In [32]:
# train second model
target_size_type2 = 1169
whole_dataset_type2 = TrainDataframeWithLabels(data_type_2, '2', target_size_type2)
model_CNN_LSTM_type2 = MultivariateCNNLSTM(3, target_size_type2, 18).to(device)

labels = data_type_2['activity'].to_list()
train_idx, val_idx = train_test_split(
    range(len(whole_dataset_type2)),
    test_size=0.2,  
    stratify=labels,
)

# create stratified datasets
train_dataset = Subset(whole_dataset_type2, train_idx)
val_dataset = Subset(whole_dataset_type2, val_idx)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_CNN_LSTM_type2.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)

best_val_loss = float('inf') 
best_model_state = copy.deepcopy(model_CNN_LSTM_type2.state_dict()) 

num_epochs = 20
for epoch in range(num_epochs):
    epoch_start_time = time.time()

    total_train_loss = 0
    train_correct = 0
    train_total = 0

    total_val_loss = 0
    val_correct = 0
    val_total = 0

    # training Phase
    model_CNN_LSTM_type2.train()  
    for batch_idx, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        normalized_inputs = (inputs - min_values_type2) / (max_values_type2 - min_values_type2 + 1e-6)
        normalized_inputs = normalized_inputs.transpose(1, 2)
        optimizer.zero_grad()
        outputs = model_CNN_LSTM_type2(normalized_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        total_train_loss += loss.item()

    # validation Phase
    model_CNN_LSTM_type2.eval() 
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            normalized_inputs = (inputs - min_values_type2) / (max_values_type2 - min_values_type2 + 1e-6)
            normalized_inputs = normalized_inputs.transpose(1, 2)
            outputs = model_CNN_LSTM_type2(normalized_inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            total_val_loss += loss.item()

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time

    train_accuracy = 100 * train_correct / train_total
    avg_train_loss = total_train_loss / len(train_dataloader)
    val_accuracy = 100 * val_correct / val_total
    avg_val_loss = total_val_loss / len(val_dataloader)
    scheduler.step(avg_val_loss)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = copy.deepcopy(model_CNN_LSTM_type2.state_dict())
        torch.save(best_model_state, 'Type2CNNLSTMModel.pth')
        print(f'Saving Best Model with loss: {avg_val_loss}')

    print("--------------")
    print(f'Epoch [{epoch + 1}/{num_epochs}], Epoch Duration: {epoch_duration:.2f} seconds')
    print(
        f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(
        f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

model_CNN_LSTM_type2 = MultivariateCNNLSTM(3, target_size_type2, 18).to(device)
model_CNN_LSTM_type2.load_state_dict(torch.load('Type2CNNLSTMModel.pth'))

Saving Best Model with loss: 1.2136844799183963
--------------
Epoch [1/20], Epoch Duration: 112.24 seconds
Epoch [1/20], Training Loss: 1.5526, Training Accuracy: 41.11%
Epoch [1/20], Validation Loss: 1.2137, Validation Accuracy: 50.98%
Saving Best Model with loss: 0.8389930594385716
--------------
Epoch [2/20], Epoch Duration: 118.09 seconds
Epoch [2/20], Training Loss: 0.9935, Training Accuracy: 61.19%
Epoch [2/20], Validation Loss: 0.8390, Validation Accuracy: 68.15%
Saving Best Model with loss: 0.6526967760240823
--------------
Epoch [3/20], Epoch Duration: 119.47 seconds
Epoch [3/20], Training Loss: 0.6953, Training Accuracy: 73.06%
Epoch [3/20], Validation Loss: 0.6527, Validation Accuracy: 73.79%
Saving Best Model with loss: 0.5871730472957879
--------------
Epoch [4/20], Epoch Duration: 118.85 seconds
Epoch [4/20], Training Loss: 0.4761, Training Accuracy: 81.82%
Epoch [4/20], Validation Loss: 0.5872, Validation Accuracy: 78.92%
Saving Best Model with loss: 0.552256779973967
-

<All keys matched successfully>

In [ ]:
# results of feature extractor without LSTM layer were better, so we did not proceed with this code